# ❌ **Cancel Culture** ❌ - **EDA Notebook**

**Who?**
>* 🏢 **Revenue Management (RM) teams** for hotel groups (corporate, franchise)
>
>
>* 🏨 On-site GMs, Sales, and Ops teams

**Why?**
>* 💰 **Revenue Management:** 
>  * Revenue optimization: Right price, right time, right customer
>    * Dynamic pricing
>    * Distribution channels
>    * Pricing per room type
>
>
>* 🤝 **Sales:**
>  * Group sales (pickup/wash)
>  * BT (performance/company for both GPP and LNR rates)
>
>
>* 🛌 **Rooms Ops:**
>  * Forecasting occupancy, arrivals, departures, stay-overs, same-day booking demand, and probability of guest relocation in the case of oversell.
>  * Determining staff schedules and periods of high demand
>
>
>* 🍰 ☕ **Food and Beverage:**
>  * Ordering food/supplies overall
>  * Scheduling staff
>  * Determining busy times (breakfast, lunch, dinner)
>    * Staffing, specific food/supplies

**What?**
>* 🧾 Dataset comprised of... 
>  * 32 different features
>    * Detailed explanation of features (and sub-categories, when appropriate) available in Readme
>  * Nearly 120,000 reservation records
>  * Source cited in Readme

❌ **How?**
>* Which models/methods? 
>* Data prep and feature engineering

---

> **Goal:** To prepare data for time series modeling and forecasting in next notebook.
>
>
> **Purpose:** to explore, clean, and organize.
>
>
> **Process:**
>
>    * Inspecting data integrity and statistics
>    * Splitting data by hotel type ("City" vs. "Resort")
>    * Filling any missing values
>    * 
>    * Save processed data for modeling notebook
>
>
> **Modeling Notebook:**
>
>    * Performing train/test split
>    * 
>    * Training the model
>    * 
>    * Evaluate performance metrics
>    * Provide final recommendations

---

# ✅ **To-Do List**

---

**Copy:**
- [ ] Imports
- [ ] Personal module
- [ ] Data
- [ ] Starter code from P4P

**Links:**
- [ ] 

---

# 📦 **Import Packages**

In [1]:
## Data Handling
import pandas as pd
import numpy as np
from scipy import stats

## Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Modeling - SKLearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn import set_config
set_config(display='diagram')


## Custom-made Functions
from bmc_functions import eda
from bmc_functions import classification as clf

## Settings
plt.style.use('seaborn-talk')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('max_rows', 100)
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

# 📥 **Read Data**

In [3]:
## Reading data
source = './data/hotel_bookings.csv'
data = pd.read_csv(source)
data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,nan,nan,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,nan,nan,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,nan,nan,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.00,nan,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.00,nan,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.00,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.00,nan,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.00,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.00,nan,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.00,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.00,nan,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.00,nan,0,Transient,104.40,0,0,Check-Out,2017-09-07


# 🔎 **EDA** 🔍

In [4]:
## Inspecting percentage of city vs. resort hotels
data['hotel'].value_counts(1)

City Hotel     0.66
Resort Hotel   0.34
Name: hotel, dtype: float64

## Splitting "City" and "Resort" 

In [5]:
## Creating subgroup for city hotels
subgroup_city = data[data['hotel'] == 'City Hotel']
subgroup_city

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
40060,City Hotel,0,6,2015,July,27,1,0,2,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.00,nan,0,Transient,0.00,0,0,Check-Out,2015-07-03
40061,City Hotel,1,88,2015,July,27,1,0,4,2,0.00,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.00,nan,0,Transient,76.50,0,1,Canceled,2015-07-01
40062,City Hotel,1,65,2015,July,27,1,0,4,1,0.00,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.00,nan,0,Transient,68.00,0,1,Canceled,2015-04-30
40063,City Hotel,1,92,2015,July,27,1,2,4,2,0.00,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.00,nan,0,Transient,76.50,0,2,Canceled,2015-06-23
40064,City Hotel,1,100,2015,July,27,2,0,2,2,0.00,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.00,nan,0,Transient,76.50,0,1,Canceled,2015-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.00,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.00,nan,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.00,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.00,nan,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.00,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.00,nan,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.00,nan,0,Transient,104.40,0,0,Check-Out,2017-09-07


In [6]:
## Creating subgroup for resort hotels
subgroup_resort = data[data['hotel'] == 'Resort Hotel']
subgroup_resort

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,nan,nan,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,nan,nan,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,nan,nan,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.00,nan,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.00,nan,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40055,Resort Hotel,0,212,2017,August,35,31,2,8,2,1.00,0,BB,GBR,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,143.00,nan,0,Transient,89.75,0,0,Check-Out,2017-09-10
40056,Resort Hotel,0,169,2017,August,35,30,2,9,2,0.00,0,BB,IRL,Direct,Direct,0,0,0,E,E,0,No Deposit,250.00,nan,0,Transient-Party,202.27,0,1,Check-Out,2017-09-10
40057,Resort Hotel,0,204,2017,August,35,29,4,10,2,0.00,0,BB,IRL,Direct,Direct,0,0,0,E,E,0,No Deposit,250.00,nan,0,Transient,153.57,0,3,Check-Out,2017-09-12
40058,Resort Hotel,0,211,2017,August,35,31,4,10,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40.00,nan,0,Contract,112.80,0,1,Check-Out,2017-09-14


# Testing Hierarchical Indexing

---

> Instead of splitting the data into two different dataframes, I may be able to create a new index for the same dataframe by splitting the "`hotel`" feature and using the two values as the first level of the row index, then the normal index values as the second level.
>
>
> This would add a layer of complexity to the data processing steps, but would reduce memory consumption and the number of dataframes.

---

In [35]:
data_mi = data
data_mi

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,nan,nan,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,nan,nan,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,nan,nan,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.00,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.00,nan,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.00,nan,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2017,August,35,30,2,5,2,0.00,0,BB,BEL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,394.00,nan,0,Transient,96.14,0,0,Check-Out,2017-09-06
119386,City Hotel,0,102,2017,August,35,31,2,5,3,0.00,0,BB,FRA,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,9.00,nan,0,Transient,225.43,0,2,Check-Out,2017-09-07
119387,City Hotel,0,34,2017,August,35,31,2,5,2,0.00,0,BB,DEU,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.00,nan,0,Transient,157.71,0,4,Check-Out,2017-09-07
119388,City Hotel,0,109,2017,August,35,31,2,5,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89.00,nan,0,Transient,104.40,0,0,Check-Out,2017-09-07


In [25]:
## Saving original index values
org_idx = data.index.values
org_idx

array([     0,      1,      2, ..., 119387, 119388, 119389], dtype=int64)

In [26]:
## Saving "hotel type" values
hotel_types = data.loc[:,'hotel'].unique()
hotel_types

array(['Resort Hotel', 'City Hotel'], dtype=object)

In [31]:
## Creating new multi-index from hotel types and original index values
multi_idx = pd.MultiIndex.from_product([hotel_types, org_idx])
multi_idx

MultiIndex([('Resort Hotel',      0),
            ('Resort Hotel',      1),
            ('Resort Hotel',      2),
            ('Resort Hotel',      3),
            ('Resort Hotel',      4),
            ('Resort Hotel',      5),
            ('Resort Hotel',      6),
            ('Resort Hotel',      7),
            ('Resort Hotel',      8),
            ('Resort Hotel',      9),
            ...
            (  'City Hotel', 119380),
            (  'City Hotel', 119381),
            (  'City Hotel', 119382),
            (  'City Hotel', 119383),
            (  'City Hotel', 119384),
            (  'City Hotel', 119385),
            (  'City Hotel', 119386),
            (  'City Hotel', 119387),
            (  'City Hotel', 119388),
            (  'City Hotel', 119389)],
           length=238780)

In [33]:
## Confirming index length doubled as expected
len(multi_idx) == 2*len(org_idx)

True

In [40]:
data_mi.reindex(labels = multi_idx, axis='index')

hotel  is_canceled  lead_time  arrival_date_year  \
Resort Hotel 0        NaN          nan        nan                nan   
             1        NaN          nan        nan                nan   
             2        NaN          nan        nan                nan   
             3        NaN          nan        nan                nan   
             4        NaN          nan        nan                nan   
...                   ...          ...        ...                ...   
City Hotel   119385   NaN          nan        nan                nan   
             119386   NaN          nan        nan                nan   
             119387   NaN          nan        nan                nan   
             119388   NaN          nan        nan                nan   
             119389   NaN          nan        nan                nan   

                    arrival_date_month  arrival_date_week_number  \
Resort Hotel 0                     NaN                       nan   
             1                     NaN                       nan   
             2                     NaN                       nan   
             3                     NaN                       nan   
             4                     NaN                       nan   
...                                ...                       ...   
City Hotel   119385                NaN                       nan   
             119386                NaN                       nan   
             119387                NaN                       nan   
             119388                NaN                       nan   
             119389                NaN                       nan   

                     arrival_date_day_of_month  stays_in_weekend_nights  \
Resort Hotel 0                             nan                      nan   
             1                             nan                      nan   
             2                             nan                      nan   
             3                             nan                      nan   
             4                             nan                      nan   
...                                        ...                      ...   
City Hotel   119385                        nan                      nan   
             119386                        nan                      nan   
             119387                        nan                      nan   
             119388                        nan                      nan   
             119389                        nan                      nan   

                     stays_in_week_nights  adults  children  babies meal  \
Resort Hotel 0                        nan     nan       nan     nan  NaN   
             1                        nan     nan       nan     nan  NaN   
             2                        nan     nan       nan     nan  NaN   
             3                        nan     nan       nan     nan  NaN   
             4                        nan     nan       nan     nan  NaN   
...                                   ...     ...       ...     ...  ...   
City Hotel   119385                   nan     nan       nan     nan  NaN   
             119386                   nan     nan       nan     nan  NaN   
             119387                   nan     nan       nan     nan  NaN   
             119388                   nan     nan       nan     nan  NaN   
             119389                   nan     nan       nan     nan  NaN   

                    country market_segment distribution_channel  \
Resort Hotel 0          NaN            NaN                  NaN   
             1          NaN            NaN                  NaN   
             2          NaN            NaN                  NaN   
             3          NaN            NaN                  NaN   
             4          NaN            NaN                  NaN   
...                     ...            ...                  ...   
City Hotel   119385     NaN            NaN             

## Reviewing Statistics

### Report - City

In [11]:
eda.report_df(subgroup_city).sort_values('null_sum', ascending=False)

,null_sum,null_pct,datatypes,num_unique,count,mean,std,min,25%,50%,75%,max
company,75641,0.95,float64,207,"3,689.00",145.27,119.77,8.00,40.00,91.00,219.00,497.00
agent,8131,0.10,float64,223,"71,199.00",28.14,56.43,1.00,9.00,9.00,17.00,509.00
country,24,0.00,object,166,nan,nan,nan,nan,nan,nan,nan,nan
children,4,0.00,float64,4,"79,326.00",0.09,0.37,0.00,0.00,0.00,0.00,3.00
adr,0,0.00,float64,5405,"79,330.00",105.30,43.60,0.00,79.20,99.90,126.00,"5,400.00"
lead_time,0,0.00,int64,453,"79,330.00",109.74,110.95,0.00,23.00,74.00,163.00,629.00
market_segment,0,0.00,object,8,nan,nan,nan,nan,nan,nan,nan,nan
meal,0,0.00,object,4,nan,nan,nan,nan,nan,nan,nan,nan
previous_bookings_not_canceled,0,0.00,int64,73,"79,330.00",0.13,1.69,0.00,0.00,0.00,0.00,72.00
previous_cancellations,0,0.00,int64,10,"79,330.00",0.08,0.42,0.00,0.00,0.00,0.00,21.00


### Report - City

In [9]:
eda.report_df(subgroup_city).sort_values('null_sum')

,null_sum,null_pct,datatypes,num_unique,count,mean,std,min,25%,50%,75%,max
total_of_special_requests,0,0.00,int64,6,"79,330.00",0.55,0.78,0.00,0.00,0.00,1.00,5.00
adults,0,0.00,int64,5,"79,330.00",1.85,0.51,0.00,2.00,2.00,2.00,4.00
stays_in_week_nights,0,0.00,int64,29,"79,330.00",2.18,1.46,0.00,1.00,2.00,3.00,41.00
arrival_date_day_of_month,0,0.00,int64,31,"79,330.00",15.79,8.73,1.00,8.00,16.00,23.00,31.00
required_car_parking_spaces,0,0.00,int64,4,"79,330.00",0.02,0.15,0.00,0.00,0.00,0.00,3.00
arrival_date_week_number,0,0.00,int64,53,"79,330.00",27.18,13.40,1.00,17.00,27.00,38.00,53.00
arrival_date_year,0,0.00,int64,3,"79,330.00","2,016.17",0.70,"2,015.00","2,016.00","2,016.00","2,017.00","2,017.00"
previous_cancellations,0,0.00,int64,10,"79,330.00",0.08,0.42,0.00,0.00,0.00,0.00,21.00
babies,0,0.00,int64,5,"79,330.00",0.00,0.08,0.00,0.00,0.00,0.00,10.00
booking_changes,0,0.00,int64,21,"79,330.00",0.19,0.61,0.00,0.00,0.00,0.00,21.00


In [9]:
eda.report_df(subgroup_city).sort_values('datatypes')

,null_sum,null_pct,datatypes,num_unique,count,mean,std,min,25%,50%,75%,max
total_of_special_requests,0,0.00,int64,6,"79,330.00",0.55,0.78,0.00,0.00,0.00,1.00,5.00
adults,0,0.00,int64,5,"79,330.00",1.85,0.51,0.00,2.00,2.00,2.00,4.00
stays_in_week_nights,0,0.00,int64,29,"79,330.00",2.18,1.46,0.00,1.00,2.00,3.00,41.00
arrival_date_day_of_month,0,0.00,int64,31,"79,330.00",15.79,8.73,1.00,8.00,16.00,23.00,31.00
required_car_parking_spaces,0,0.00,int64,4,"79,330.00",0.02,0.15,0.00,0.00,0.00,0.00,3.00
arrival_date_week_number,0,0.00,int64,53,"79,330.00",27.18,13.40,1.00,17.00,27.00,38.00,53.00
arrival_date_year,0,0.00,int64,3,"79,330.00","2,016.17",0.70,"2,015.00","2,016.00","2,016.00","2,017.00","2,017.00"
previous_cancellations,0,0.00,int64,10,"79,330.00",0.08,0.42,0.00,0.00,0.00,0.00,21.00
babies,0,0.00,int64,5,"79,330.00",0.00,0.08,0.00,0.00,0.00,0.00,10.00
booking_changes,0,0.00,int64,21,"79,330.00",0.19,0.61,0.00,0.00,0.00,0.00,21.00


---

**Reviewing Reports - Missing Values**

> Based on the post-split results, I see that the "`company`" feature is missing 95% of the results and the "`agent`" feature is missing 10%.

**"`Company`" and "`Agent`" Features**

> **Due to the large number of missing values for "`company`," I will drop that column.** As noted in the data's documentation, any missing values are intentional representations of features that were not applicable to a reservation. **Since the missing values are valid, I will keep the "`agent`" column and will fill the missing values with an "N/A" value to represent the lack of a value.**

**"`Country`" and "`Children`" Features**

> The remaining two features with missing values are "`country`" and "`children`." Due to the small number of missing values for each, I will simply impute the most frequent value, respectively.

---

### Report - City - Data Types

In [13]:
eda.report_df(subgroup_city).sort_values('datatypes', ascending=False)

,null_sum,null_pct,datatypes,num_unique,count,mean,std,min,25%,50%,75%,max
distribution_channel,0,0.00,object,5,nan,nan,nan,nan,nan,nan,nan,nan
country,24,0.00,object,166,nan,nan,nan,nan,nan,nan,nan,nan
reserved_room_type,0,0.00,object,8,nan,nan,nan,nan,nan,nan,nan,nan
reservation_status_date,0,0.00,object,864,nan,nan,nan,nan,nan,nan,nan,nan
arrival_date_month,0,0.00,object,12,nan,nan,nan,nan,nan,nan,nan,nan
reservation_status,0,0.00,object,3,nan,nan,nan,nan,nan,nan,nan,nan
meal,0,0.00,object,4,nan,nan,nan,nan,nan,nan,nan,nan
assigned_room_type,0,0.00,object,9,nan,nan,nan,nan,nan,nan,nan,nan
market_segment,0,0.00,object,8,nan,nan,nan,nan,nan,nan,nan,nan
hotel,0,0.00,object,1,nan,nan,nan,nan,nan,nan,nan,nan


### Report - Resort - Data Types

In [15]:
eda.report_df(subgroup_resort).sort_values('datatypes', ascending=False)

,null_sum,null_pct,datatypes,num_unique,count,mean,std,min,25%,50%,75%,max
distribution_channel,0,0.00,object,4,nan,nan,nan,nan,nan,nan,nan,nan
country,464,0.01,object,125,nan,nan,nan,nan,nan,nan,nan,nan
reserved_room_type,0,0.00,object,10,nan,nan,nan,nan,nan,nan,nan,nan
reservation_status_date,0,0.00,object,913,nan,nan,nan,nan,nan,nan,nan,nan
arrival_date_month,0,0.00,object,12,nan,nan,nan,nan,nan,nan,nan,nan
reservation_status,0,0.00,object,3,nan,nan,nan,nan,nan,nan,nan,nan
meal,0,0.00,object,5,nan,nan,nan,nan,nan,nan,nan,nan
assigned_room_type,0,0.00,object,11,nan,nan,nan,nan,nan,nan,nan,nan
market_segment,0,0.00,object,6,nan,nan,nan,nan,nan,nan,nan,nan
hotel,0,0.00,object,1,nan,nan,nan,nan,nan,nan,nan,nan


## Dropping "Company" Column

In [ ]:
# Dropping "company" column (95% missing values) , "hotel" column (only 1 value)
subgroup_city.drop(columns = ['company', 'hotel'], inplace=True)
subgroup_city

In [ ]:
# Dropping "company" column (95% missing values) , "hotel" column (only 1 value)
subgroup_resort.drop(columns = ['company', 'hotel'], inplace=True)
subgroup_resort

### City

In [ ]:
eda.report_df(subgroup_city).sort_values('null_sum', ascending=False)

In [ ]:
# Dropping "company" column (95% missing values) , "hotel" column (only 1 value)
subgroup_city.drop(columns = ['company', 'hotel'], inplace=True)
subgroup_city

In [ ]:
## Identifying columns for feature-by-feature EDA
subgroup_city.columns

### Resort

In [ ]:
eda.report_df(subgroup_resort).sort_values('null_sum', ascending=False)

In [ ]:
subgroup_resort.columns

# **EDA - Features**

## reservation_status

### City

In [ ]:
subgroup_city['reservation_status'].value_counts(1, dropna=False)

### Resort

In [ ]:
subgroup_resort['reservation_status'].value_counts(1, dropna=False)

### ❌ Binarizing - New Feature

### City

In [ ]:
## Changing no-show values to "canceled"
subgroup_city['reservation_status'].replace('No-Show', 'Canceled',
                                            inplace=True)

In [ ]:
## Confirming the change
'No-Show' in subgroup_city['reservation_status']

In [ ]:
## Inspecting the updated target classes
subgroup_city['reservation_status'].value_counts(1, dropna=False)

In [ ]:
cond = [subgroup_city['reservation_status'] == 'Check-Out',
       subgroup_city['reservation_status'] == 'Canceled']

choice = [0, 1]

subgroup_city['res_status_binary'] = np.select(cond, choice, 0)
subgroup_city['res_status_binary']

In [ ]:
subgroup_city['res_status_binary'].value_counts(1)

#### Resort

In [ ]:
## Changing no-show values to "canceled"
subgroup_resort['reservation_status'].replace('No-Show', 'Canceled',
                                            inplace=True)

In [ ]:
## Confirming the change
'No-Show' in subgroup_resort['reservation_status']

In [ ]:
## Inspecting the updated target classes
subgroup_resort['reservation_status'].value_counts(1, dropna=False)

In [ ]:
cond = [subgroup_resort['reservation_status'] == 'Check-Out',
       subgroup_resort['reservation_status'] == 'Canceled']

choice = [0, 1]

subgroup_resort['res_status_binary'] = np.select(cond, choice, 0)
subgroup_resort['res_status_binary']

In [ ]:
subgroup_resort['res_status_binary'].value_counts(1)

## is_canceled

### City

In [ ]:
subgroup_city['is_canceled'].value_counts(1, dropna=False)

In [ ]:
## Visualizing results
fig, ax = plt.subplots()
subgroup_city['is_canceled'].value_counts(1, dropna=False).plot(kind='barh',
                                                                ax=ax)
ax.set_yticklabels(['Not Cancelled', 'Cancelled'])
ax.set_xlabel('Percentage')
ax.set_ylabel('Status')
ax.set_title('Reservation Statuses');

### Resort

In [ ]:
subgroup_resort['is_canceled'].value_counts(1, dropna=False)

In [ ]:
## Visualizing results
fig, ax = plt.subplots()
subgroup_resort['is_canceled'].value_counts(1, dropna=False).plot(kind='barh',
                                                                ax=ax)
ax.set_yticklabels(['Not Cancelled', 'Cancelled'])
ax.set_xlabel('Percentage')
ax.set_ylabel('Status')
ax.set_title('Reservation Statuses');

## lead_time - Fix legend labels!

### City

In [ ]:
subgroup_city['lead_time'].describe()

In [ ]:
# fig, ax = plt.subplots(figsize=(5,10))
# subgroup_city['lead_time'].plot(kind='box', ax=ax)

# ax.set_title('Lead Time');

In [ ]:
fig = px.box(subgroup_city, y="lead_time", title='Overview of Lead Time',
             width=600, color='reservation_status',
             labels = {'lead_time': 'Lead Time (Days)'},
            category_orders = {'lead_time': 'Lead Time (Days)',
                               'reservation_status': ['Check-Out', 'Canceled']})

fig.show()

### Resort

In [ ]:
subgroup_resort['lead_time'].describe()

In [ ]:
# fig, ax = plt.subplots(figsize=(5,10))
# subgroup_resort['lead_time'].plot(kind='box', ax=ax)

# ax.set_title('Lead Time');

In [ ]:
fig = px.box(subgroup_resort, y="lead_time", title='Overview of Lead Time',
             width=600, color='reservation_status',
             labels = {'lead_time': 'Lead Time (Days)'},
            category_orders = {'lead_time': 'Lead Time (Days)',
                               'reservation_status': ['Check-Out', 'Canceled']})

fig.show()

## Arrival Date as Full Datetime

### City

In [ ]:
## Converting from month, day of month, and year to a single datetime column
subgroup_city['arrival_date'] = subgroup_city['arrival_date_month'] +' '+ \
                                subgroup_city['arrival_date_day_of_month']\
                                .astype(str) +', '+ \
                                subgroup_city['arrival_date_year'].astype(str)
subgroup_city['arrival_date'] = pd.to_datetime(subgroup_city['arrival_date'])
subgroup_city['arrival_date']

In [ ]:
fig = px.box(subgroup_city, y="lead_time", title='Lead Times',
             width=600, color='reservation_status',
             labels = {'lead_time': 'Lead Time (Days)'},
            category_orders = {'lead_time': 'Lead Time (Days)',
                               'reservation_status': ['Check-Out', 'Canceled']})

fig.show()

In [ ]:
fig = px.histogram(subgroup_city,'lead_time', marginal = 'box',
                   color='reservation_status',
                   labels={'lead_time': 'Lead Time (Days)'}, 
                   title="Lead Times", nbins=40)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
fig = px.scatter(subgroup_city, x="reservation_status", y="lead_time",
                 marginal_y="box",marginal_x="histogram",
                 color='reservation_status')
fig.show()

In [ ]:
# ## Try again with select features as dimensions
# fig = px.scatter_matrix(subgroup_city,
# #     dimensions=["sepal_width", "sepal_length", "petal_width", "petal_length"],
#     color="reservation_status")
# fig.show()

### Resort

In [ ]:
## Converting from month, day of month, and year to a single datetime column
subgroup_resort['arrival_date'] = subgroup_resort['arrival_date_month'] +' '+ \
                                subgroup_resort['arrival_date_day_of_month']\
                                .astype(str) +', '+ \
                                subgroup_resort['arrival_date_year'].astype(str)
subgroup_resort['arrival_date'] = pd.to_datetime(subgroup_resort['arrival_date'])
subgroup_resort['arrival_date']

In [ ]:
fig = px.box(subgroup_resort, y="lead_time", title='Lead Times',
             width=600, color='reservation_status',
             labels = {'lead_time': 'Lead Time (Days)'},
            category_orders = {'lead_time': 'Lead Time (Days)',
                               'reservation_status': ['Check-Out', 'Canceled']})

fig.show()

In [ ]:
fig = px.histogram(subgroup_resort,'lead_time', marginal = 'box',
                   color='reservation_status',
                   labels={'lead_time': 'Lead Time (Days)'}, 
                   title="Lead Times", nbins=40)
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:
fig = px.scatter(subgroup_resort, x="reservation_status", y="lead_time",
                 marginal_y="box",marginal_x="histogram",
                 color='reservation_status')
fig.show()

In [ ]:
# ## Try again with select features as dimensions
# fig = px.scatter_matrix(subgroup_resort,
# #     dimensions=["sepal_width", "sepal_length", "petal_width", "petal_length"],
#     color="reservation_status")
# fig.show()

## stays_in_weekend_nights

In [ ]:
subgroup_city['stays_in_weekend_nights'].value_counts(1)

In [ ]:
fig = px.histogram(subgroup_city,'stays_in_weekend_nights', marginal = 'box',
                   labels={'stays_in_weekend_nights': 'Number of Weekend Nights'}, 
                   title="Weekend Stays", color='reservation_status', nbins=15)
fig.update_layout(bargap=0.2)
fig.show()

## stays_in_week_nights

In [ ]:
subgroup_city['stays_in_week_nights'].value_counts(1)

In [ ]:
subgroup_city['stays_in_week_nights'].value_counts(1)[:6]

In [ ]:
fig = px.histogram(subgroup_city,'stays_in_week_nights', marginal = 'box',
                   labels={'stays_in_week_nights': 'Number of Week Nights'}, 
                   title="Weekday Stays", color='reservation_status', nbins=40)
fig.update_layout(bargap=0.2)
fig.show()

## Adults

In [ ]:
subgroup_city['adults'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'adults',
                   labels={'adults': 'Number of Adults'},
                   title="Number of Adults", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## Children

In [ ]:
subgroup_city['children'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'children', marginal = 'box',
                   labels={'children': 'Number of Children'}, 
                   title="Number of Children", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## babies

In [ ]:
subgroup_city['babies'].value_counts(dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'babies', marginal = 'box',
                   labels={'babies': 'Number of Babies'}, 
                   title="Number of Babies", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## meal

In [ ]:
subgroup_city['meal'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'meal',labels={'meal': 'Types of Meals'}, 
                   title="Dining with Us?", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## country

In [ ]:
subgroup_city['country'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'country',
                   labels={'country': 'Country of Origin'}, 
                   title="'Where's Home?'", color='reservation_status', nbins=15)
fig.update_layout(bargap=0.2)
fig.show()

## market_segment

In [ ]:
subgroup_city['market_segment'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'market_segment',
                   labels={'market_segment': 'Market Segment'}, 
                   title="Segmentation", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## distribution_channel

In [ ]:
subgroup_city['distribution_channel'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'distribution_channel',
                   labels={'distribution_channel': 'Channel'}, 
                   title="Distribution Channels", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## is_repeated_guest

In [ ]:
subgroup_city['is_repeated_guest'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'is_repeated_guest',
                   labels={'is_repeated_guest': 'Repeat Status'}, 
                   title="Welcome Back!", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## previous_cancellations

In [ ]:
subgroup_city['previous_cancellations'].describe()

In [ ]:
fig = px.histogram(subgroup_city,'previous_cancellations',
                   labels={'previous_cancellations': 'Number of Cancellations'}, 
                   title="Previous Cancellations", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## previous_bookings_not_canceled

In [ ]:
subgroup_city['previous_bookings_not_canceled'].describe()

In [ ]:
fig = px.histogram(subgroup_city,'previous_bookings_not_canceled', marginal = 'box',
                   labels={'previous_bookings_not_canceled': 'Number of Prior Stays'}, 
                   title="Prior Stays", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

##  reserved_room_type

In [ ]:
subgroup_city['reserved_room_type'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'reserved_room_type',
                   labels={'reserved_room_type': 'Room Type'}, 
                   title="Reserved Room Types", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## assigned_room_type

In [ ]:
subgroup_city['assigned_room_type'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'assigned_room_type',
                   labels={'assigned_room_type': 'Assigned Room Type'}, 
                   title="Assigned Room Types", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## booking_changes

In [ ]:
subgroup_city['booking_changes'].value_counts(1)

In [ ]:
fig = px.histogram(subgroup_city,'booking_changes', marginal = 'box',
                   labels={'booking_changes': 'Number of Changes'}, 
                   title="Booking Changes", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## deposit_type

In [ ]:
subgroup_city['deposit_type'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'deposit_type',
                   labels={'deposit_type': 'Type'}, 
                   title="Deposit Types", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## agent

In [ ]:
subgroup_city['agent'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'agent', marginal = 'box',
                   labels={'agent': 'Booking Agent ID Number'}, 
                   title="Bookings per Agent", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## days_in_waiting_list

In [ ]:
subgroup_city['days_in_waiting_list'].describe()

In [ ]:
fig = px.histogram(subgroup_city,'days_in_waiting_list', marginal = 'box',
                   labels={'days_in_waiting_list': 'Number of Days'}, 
                   title="Days on Waiting List", color='reservation_status', nbins=15)
fig.update_layout(bargap=0.2)
fig.show()

## customer_type

In [ ]:
subgroup_city['customer_type'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'customer_type',
                   labels={'customer_type': 'Reservation Type'}, 
                   title="Reservation Types", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## adr

In [ ]:
subgroup_city['adr'].describe()

In [ ]:
fig = px.histogram(subgroup_city,'adr', marginal = 'box',
                   labels={'adr': 'Rate'}, title="Average Daily Rate (ADR)",
                   color='reservation_status', nbins=15)
fig.update_layout(bargap=0.2)
fig.show()

## required_car_parking_spaces

In [ ]:
subgroup_city['required_car_parking_spaces'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'required_car_parking_spaces',
                   labels={'required_car_parking_spaces': 'Number of Cars'}, 
                   title="Number of Cars", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## total_of_special_requests

In [ ]:
subgroup_city['total_of_special_requests'].value_counts(1, dropna=False)

In [ ]:
fig = px.histogram(subgroup_city,'total_of_special_requests',
                   labels={'total_of_special_requests': 'Number of Requests'}, 
                   title="Number of Special Requests", color='reservation_status')
fig.update_layout(bargap=0.2)
fig.show()

## reservation_status_date

In [ ]:
subgroup_city['reservation_status_date']

# 📅 **Setting Datetime Index**

In [ ]:
city_ts = subgroup_city.set_index('arrival_date')
city_ts

In [ ]:
resort_ts = subgroup_resort.set_index('arrival_date')
resort_ts